# Reconocimiento de imágenes con YOLO9000
**Universidad de Chile**<br>
**Facultad de Ciencias, Física y Matemáticas**<br>
**Departamento de Ciencias de Computación**<br>
**CC5509 - Reconocimiento de Patrones**<br>
<br>
*Profesores:*<br>
**José M. Saavedra R.**<br>
**Mauricio Cerda Villablanca**<br>
<br>
*Ayudante:*<br>
**Camila Álvarez I.**<br>
<br>
*Alumnos:*<br>
**Cristobal Dotte**<br>
**Gabriel De La Parra**<br>

## Introducción

En este proyecto, se trabajó con [YOLO9000](https://pjreddie.com/darknet/yolo/) para hacer detección y clasificación de objetos en imágenes.

El [paper](https://arxiv.org/abs/1612.08242) fue escrito por Joseph Redmon y Ali Farhadi en Diciembre del 2016, por lo que se puede considerar como un trabajo en el *state-of-the-art*.

YOLO (You Only Look Once) en la versión YOLO9000 propone varias mejoras al sistema YOLO original de los mismos autores. Esta versión lleva el nombre 9000 ya que puede detectar sobre 9000 clases de objetos. Las mejores a esta versión provienen de sugerencias de otros autores y técnicas implementadas por otros sistemas. 

Uno de los aspectos más importantes de YOLO, es que puede correr a 67FPS con un mAP (*Mean Average Precision*) del 76.8 en VOC 2007. Este resultado es mucho mejor que otras técnicas existentes actualmente y a una velocidad mucho mayor. Los detalles sobre la comparación se pueden encontrar en el Paper.

El objetivo de este proyecto será realizar detección sobre carteles de precios en imágenes de frutas y verduras de ferias libres y la vega.

## Procedimiento

Se realizó el siguiente procedimiento para lograr el reconocimiento y clasificación de las imágenes:

1. Construcción del dataset
    1. Obtención de imágenes
    2. Preprocesamiento
    2. Etiquetado
    3. Adaptación de etiquetas
2. Entrenamiento
    1. Descarga YOLO
    2. Compilación
    3. Pruebas
    4. Archivos de configuración
    4. Train y training set
    5. Entrenamiento
3. Pruebas
    1. Pruebas
    2. Resultados
4. Conclusiones

## Dataset

En este proyecto, se trabajará con un dataset generado desde cero para este proyecto. El dataset está compuesto por 878 imágenes. Las fotos fueron etiquetadas ocupando la herramienta [LabelImg](https://github.com/tzutalin/labelImg). Dicha herramienta tiene un formato de salida que no es el mismo que requiere YOLO. Fue necesario crear un script para convertir los archivos de salida.

### Obtención de imágenes

Las imágenes fueron tomadas en la vega y en dos ferias libres de Santiago de Chile.

A continuación se presenta un sample de algunas de estas fotos.

In [20]:
from IPython.display import Image, HTML, display
from glob import glob
imagesList=''.join( ["<img style='width: 300px; margin: 2px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                 for s in sorted(glob('feria labeled/*.jpg')[1:100:10]) ])
display(HTML(imagesList))

## Preprocesamiento

Cada imagen tiene originalmente nombres como: `Foto 03-11-17 14 47 04.jpg` y tiene una resolución de `3264x2448`

### Resize
Si bien la resolución inicial tiene más detalle, no es necesario tanta resolución para entrenar las imagenes, debido a que YOLO hace transformación de las imagenes. Otro problema de trabajar con las imagenes a tan gran resolución, es que copiarlas al servidor ocupa mucho más recursos/tiempo. El set original tiene 2.5gb. El nuevo, 90 mb. 

Como se verá más adelante, se entrenó con la resolución de `418x418`. Se redujo en tamaño al `20%`, que corresponde a `693x490`.

``` bash
$ for a in *.jpg; do convert "$a" -resize 20% resized/"$a"; done
```

![resize](images/resize.png)

### Rename
Los nombres de las imagenes no influyen mucho, sin embargo son maś complicados para seleccionar al momento de usarlos para testear. Por lo mismo, se renombraron las imagenes de forma secuencial. El código que se utilizó es el siguiente.

``` bash
$ ls | cat -n | while read n f; do mv "$f" "$n.jpg"; done
```

![rename](images/rename.png)

## Etiquetado

Se utilizó la herramienta [LabelImg](https://github.com/tzutalin/labelImg) para etiquetar las imagenes. A continuación se presenta una muestra de la interfáz gráfica de la herramienta:

![labelimg](images/labelimg.png)

Uno de los beneficios de utilizar esta herramienta, es que se pueden utilizar distintas clases para las imágenes. Adicionalmente, la herramienta tiene comandos:
- `w`: crear un cuadro
- `a`: imagen anterior
- `d`: imagen siguiente

Por cada imagen, se crea un archivo de formato xml. El archivo tiene el siguiente contenido:

In [27]:
!head -25 1.xml

<annotation>
	<folder>feria</folder>
	<filename>1.jpg</filename>
	<path>/home/clgadel/labeler/feria/1.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>653</width>
		<height>490</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>manzana</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>192</xmin>
			<ymin>285</ymin>
			<xmax>227</xmax>
			<ymax>317</ymax>
		</bndbox>
	</object>


## Adaptación de etiquetas

Para el procesamiento de las etiquetas utilizaremos la librería [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/).

In [28]:
from bs4 import BeautifulSoup

### Lectura de etiquetas

In [29]:
soup = BeautifulSoup(open("1.xml", "r"), "xml")

### Preview

In [34]:
print(str(soup)[0:496])

<?xml version="1.0" encoding="utf-8"?>
<annotation>
<folder>feria</folder>
<filename>1.jpg</filename>
<path>/home/clgadel/labeler/feria/1.jpg</path>
<source>
<database>Unknown</database>
</source>
<size>
<width>653</width>
<height>490</height>
<depth>3</depth>
</size>
<segmented>0</segmented>
<object>
<name>manzana</name>
<pose>Unspecified</pose>
<truncated>0</truncated>
<difficult>0</difficult>
<bndbox>
<xmin>192</xmin>
<ymin>285</ymin>
<xmax>227</xmax>
<ymax>317</ymax>
</bndbox>
</object>



### Ubicar Etiquetas

In [35]:
objects = soup.find_all("object")

In [36]:
objects[1]

<object>
<name>manzana</name>
<pose>Unspecified</pose>
<truncated>0</truncated>
<difficult>0</difficult>
<bndbox>
<xmin>202</xmin>
<ymin>305</ymin>
<xmax>240</xmax>
<ymax>340</ymax>
</bndbox>
</object>

### Nombre de la etiqueta

In [37]:
obj = objects[0]
print(obj.find("name").text)

manzana


### Coordenadas de la etiqueta

In [38]:
obj = objects[0]
print(obj.find("name").text, obj.bndbox.xmin.text, obj.bndbox.ymin.text)

manzana 192 285


### Tamaño de la imagen

In [39]:
size = soup.annotation.size
size

<size>
<width>653</width>
<height>490</height>
<depth>3</depth>
</size>

In [40]:
print(size.width.text, size.height.text)

653 490


### Conversión al formato necesario

YOLO necesita de un formato necesario para trabajar. Este formato, según la página debe ser del siguiente tipo:

```
Generate Labels for VOC

Now we need to generate the label files that Darknet uses. Darknet wants a .txt file for each image with a line for each ground truth object in the image that looks like:

<object-class> <x> <y> <width> <height>
Where x, y, width, and height are relative to the image's width and height.
```

Existe [un repositorio](https://github.com/Guanghan/darknet/blob/master/scripts/convert.py) para hacer la conversión de las imágenes, pero funciona con otro tipo y fue necesario adaptarlo. A continuación los cambios.

In [45]:
size = soup.annotation.size
dw, dh = int(size.width.text), int(size.height.text)
print("Size:\t\t\t", dw, dh)
dw, dh = 1./int(size.width.text), 1./int(size.height.text)
print("Size porcentual:\t", dw, dh)
print()
obj = objects[0]
print("Bounding box:\t\t", obj.bndbox.xmin.text, obj.bndbox.ymin.text, obj.bndbox.xmax.text, obj.bndbox.ymax.text)
x, y = (int(obj.bndbox.xmin.text) + int(obj.bndbox.xmax.text))/2, (int(obj.bndbox.ymin.text) + int(obj.bndbox.ymax.text))/2
w, h = int(obj.bndbox.xmax.text)-int(obj.bndbox.xmin.text), int(obj.bndbox.ymax.text)-int(obj.bndbox.ymin.text)

x = x*dw
y = y*dh
w = w*dw
h = h*dh

print("Dimensiones YOLO:\t", x, y, w, h)

Size:			 653 490
Size porcentual:	 0.0015313935681470138 0.0020408163265306124

Bounding box:		 192 285 227 317
Dimensiones YOLO:	 0.3208269525267994 0.6142857142857143 0.053598774885145486 0.0653061224489796


### Crear etiquetas

In [46]:
size = soup.annotation.size
dw, dh = int(size.width.text), int(size.height.text)
dw, dh = 1./int(size.width.text), 1./int(size.height.text)

for obj in objects:
    x, y = (int(obj.bndbox.xmin.text) + int(obj.bndbox.xmax.text))/2, (int(obj.bndbox.ymin.text) + int(obj.bndbox.ymax.text))/2
    w, h = int(obj.bndbox.xmax.text)-int(obj.bndbox.xmin.text), int(obj.bndbox.ymax.text)-int(obj.bndbox.ymin.text)

    x = x*dw
    y = y*dh
    w = w*dw
    h = h*dh
    print(obj.find("name").text, x, y, w, h)

manzana 0.3208269525267994 0.6142857142857143 0.053598774885145486 0.0653061224489796
manzana 0.33843797856049007 0.6581632653061225 0.05819295558958652 0.07142857142857144
manzana 0.34915773353751917 0.6948979591836736 0.06125574272588055 0.0673469387755102
manzana 0.26339969372128635 0.6948979591836736 0.06738131699846861 0.0673469387755102
manzana 0.2840735068912711 0.7459183673469388 0.06278713629402757 0.07959183673469389
manzana 0.33843797856049007 0.7622448979591837 0.06431852986217458 0.07142857142857144
manzana 0.3973966309341501 0.7326530612244898 0.05972434915773354 0.07346938775510205
manzana 0.20444104134762633 0.7438775510204082 0.06584992343032159 0.06326530612244899
manzana 0.4004594180704441 0.653061224489796 0.05053598774885146 0.07346938775510205
manzana 0.35298621745788666 0.5979591836734695 0.03215926493108729 0.0489795918367347
manzana 0.37059724349157736 0.5877551020408164 0.030627871362940276 0.04489795918367347
manzana 0.4058192955589587 0.5836734693877551 0.03

### Guardar etiquetas

In [47]:
with open("1.txt", "w") as f: 
    for obj in objects:
        x, y = (int(obj.bndbox.xmin.text) + int(obj.bndbox.xmax.text))/2, (int(obj.bndbox.ymin.text) + int(obj.bndbox.ymax.text))/2
        w, h = int(obj.bndbox.xmax.text)-int(obj.bndbox.xmin.text), int(obj.bndbox.ymax.text)-int(obj.bndbox.ymin.text)

        x = x*dw
        y = y*dh
        w = w*dw
        h = h*dh
        f.write(" ".join((obj.find("name").text, str(x), str(y), str(w), str(h))))
        f.write("\n")
        
!cat 1.txt

manzana 0.3208269525267994 0.6142857142857143 0.053598774885145486 0.0653061224489796
manzana 0.33843797856049007 0.6581632653061225 0.05819295558958652 0.07142857142857144
manzana 0.34915773353751917 0.6948979591836736 0.06125574272588055 0.0673469387755102
manzana 0.26339969372128635 0.6948979591836736 0.06738131699846861 0.0673469387755102
manzana 0.2840735068912711 0.7459183673469388 0.06278713629402757 0.07959183673469389
manzana 0.33843797856049007 0.7622448979591837 0.06431852986217458 0.07142857142857144
manzana 0.3973966309341501 0.7326530612244898 0.05972434915773354 0.07346938775510205
manzana 0.20444104134762633 0.7438775510204082 0.06584992343032159 0.06326530612244899
manzana 0.4004594180704441 0.653061224489796 0.05053598774885146 0.07346938775510205
manzana 0.35298621745788666 0.5979591836734695 0.03215926493108729 0.0489795918367347
manzana 0.37059724349157736 0.5877551020408164 0.030627871362940276 0.04489795918367347
manzana 0.4058192955589587 0.5836734693877551 0.03

### Archivos XML

In [48]:
import os

In [49]:
xml_files = [f for f in os.listdir("labels") if f.endswith('.xml')]
xml_files[0:10]

['163.xml',
 '156.xml',
 '166.xml',
 '236.xml',
 '47.xml',
 '272.xml',
 '134.xml',
 '372.xml',
 '25.xml',
 '385.xml']

### Todos a TXT

In [50]:
xml_files = [f for f in os.listdir("labels") if f.endswith('.xml')]
for xml_file in xml_files:
    soup = BeautifulSoup(open("labels/"+xml_file, "r"), "xml")
    size = soup.annotation.size
    dw, dh = int(size.width.text), int(size.height.text)
    dw, dh = 1./int(size.width.text), 1./int(size.height.text)
    
    objects = soup.find_all("object")
    output_file = xml_file.replace(".xml", ".txt")
    with open("txt_labels/"+output_file, "w") as f: 
        for obj in objects:
            x, y = (int(obj.bndbox.xmin.text) + int(obj.bndbox.xmax.text))/2, (int(obj.bndbox.ymin.text) + int(obj.bndbox.ymax.text))/2
            w, h = int(obj.bndbox.xmax.text)-int(obj.bndbox.xmin.text), int(obj.bndbox.ymax.text)-int(obj.bndbox.ymin.text)

            x = x*dw
            y = y*dh
            w = w*dw
            h = h*dh
            f.write(" ".join(("0", str(x), str(y), str(w), str(h))))
            f.write("\n")

## Entrenamiento